In [156]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
from tactis.model.tactis import TACTiS

In [157]:
# Load and prepare data
df = pd.read_csv("Apr_2023.csv", parse_dates=['Timestamp']).set_index('Timestamp').iloc[2000:]
df_filled = df.ffill().bfill()  # Fix deprecated warning
data_mean, data_std = df_filled.mean(), df_filled.std() + 1e-8
df_normalized = (df_filled - data_mean) / data_std

In [145]:
df.head(5)

,Battery_Active_Power,Battery_Active_Power_Set_Response,PVPCS_Active_Power,GE_Body_Active_Power,GE_Active_Power,GE_Body_Active_Power_Set_Response,FC_Active_Power_FC_END_Set,FC_Active_Power,FC_Active_Power_FC_end_Set_Response,Island_mode_MCCB_Active_Power,MG-LV-MSB_AC_Voltage,Receiving_Point_AC_Voltage,Island_mode_MCCB_AC_Voltage,Island_mode_MCCB_Frequency,MG-LV-MSB_Frequency,Inlet_Temperature_of_Chilled_Water,Outlet_Temperature
Timestamp,,,,,,,,,,,,,,,,,
2023/04/01 05:33:21,0.0,0.0,0.0,131.0,134.300003,122.0,40.0,38.0,40.0,-131.0,489.0,487.0,489.0,59.970001,59.970001,12.6,14.1
2023/04/01 05:33:31,0.1,0.0,0.0,112.0,128.800003,122.0,40.0,38.0,40.0,-135.0,489.0,487.0,489.0,59.980000,59.980000,12.7,14.1
2023/04/01 05:33:41,-0.1,0.0,0.0,124.0,97.500000,122.0,40.0,38.0,40.0,-127.0,489.0,487.0,489.0,59.980000,59.980000,12.7,14.1
2023/04/01 05:33:51,0.0,0.0,0.0,111.0,136.500000,122.0,40.0,38.0,40.0,-126.0,489.0,487.0,489.0,59.980000,59.980000,12.7,14.1
2023/04/01 05:34:01,-0.2,0.0,0.0,110.0,112.300003,122.0,40.0,38.0,40.0,-133.0,489.0,487.0,489.0,59.980000,59.970001,12.7,14.1


In [146]:
len(df.columns)

17

In [147]:
len(data_values)

17

In [158]:
data_values = df_normalized.values.T  # [num_series, timesteps]
train_size = int(data_values.shape[1] * 0.8)
train_data = data_values[:, :train_size]
test_data = data_values[:, train_size:]
hist_length, pred_length = 200, 100
batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [136]:
# hist_length = 24: Input context window of 24 time steps that the model uses to understand past patterns
# pred_length = 1: How many future time steps to forecast after the history window
# batch_size = 16: Number of training examples processed simultaneously for computational efficiency

In [149]:
train_data.shape

(17, 212499)

In [159]:
model = TACTiS(
    num_series=train_data.shape[0],
    flow_series_embedding_dim=4,
    copula_series_embedding_dim=4,
    flow_input_encoder_layers=1,
    copula_input_encoder_layers=1,
    input_encoding_normalization=True,
    data_normalization="standardization",
    loss_normalization="series",
    positional_encoding={"dropout": 0.1},
    flow_temporal_encoder={
        "attention_layers": 1, "attention_heads": 1,
        "attention_dim": 8, "attention_feedforward_dim": 8, "dropout": 0.1,
    },
    copula_temporal_encoder={
        "attention_layers": 1, "attention_heads": 1,
        "attention_dim": 8, "attention_feedforward_dim": 8, "dropout": 0.1,
    },
    copula_decoder={
        "min_u": 0.05, "max_u": 0.95,
        "attentional_copula": {
            "attention_heads": 1, "attention_layers": 1, "attention_dim": 8,
            "mlp_layers": 1, "mlp_dim": 16, "resolution": 10,
            "dropout": 0.1, "activation_function": "relu",
        },
        "dsf_marginal": {
            "mlp_layers": 1, "mlp_dim": 16,
            "flow_layers": 1, "flow_hid_dim": 8,
        },
    },
).to(device)

In [160]:
def train_phase(model, optimizer, data, hist_length, pred_length, batch_size, 
                num_epochs, phase, clip_value=1.0):
    max_idx = data.shape[1] - (hist_length + pred_length)
    
    for epoch in range(num_epochs):
        epoch_loss, valid_batches = 0.0, 0
        num_batches = 5
        
        for _ in range(num_batches):
            # Create batch data
            indices = np.random.randint(0, max_idx, batch_size)
            hist_values = np.array([data[:, i:i+hist_length] for i in indices])
            pred_values = np.array([data[:, i+hist_length:i+hist_length+pred_length] for i in indices])
            
            hist_value = torch.tensor(hist_values, dtype=torch.float32).to(device)
            pred_value = torch.tensor(pred_values, dtype=torch.float32).to(device)
            hist_time = torch.arange(0, hist_length, device=device)[None, :].expand(batch_size, -1)
            pred_time = torch.arange(hist_length, hist_length + pred_length, device=device)[None, :].expand(batch_size, -1)
            
            try:
                optimizer.zero_grad()
                _ = model.loss(hist_time, hist_value, pred_time, pred_value)
                
                loss = -model.marginal_logdet if phase == 1 else model.copula_loss
                loss_avg = loss.mean()
                
                if torch.isnan(loss_avg):
                    continue
                    
                loss_avg.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
                optimizer.step()
                
                epoch_loss += loss_avg.item()
                valid_batches += 1
                
            except Exception as e:
                continue
        
        avg_loss = epoch_loss / valid_batches if valid_batches > 0 else float('nan')
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.6f}, Valid Batches: {valid_batches}/{num_batches}")

In [161]:
def train_model():
    # Phase 1: Train marginals
    optimizer = optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-6)
    print("Phase 1 Training (Marginals)")
    train_phase(model, optimizer, train_data, hist_length, pred_length, batch_size, num_epochs=5, phase=1)
    
    # Phase 2: Train copula
    model.set_stage(2)
    model.initialize_stage2()
    
    # Only optimize copula parameters
    params_to_optimize = [p for n, p in model.named_parameters() 
                          if any(x in n for x in ["copula", "decoder.copula"])]
    optimizer = optim.AdamW(params_to_optimize, lr=5e-4, weight_decay=1e-6)
    
    print("\nPhase 2 Training (Copula)")
    train_phase(model, optimizer, train_data, hist_length, pred_length, batch_size, num_epochs=10, phase=2)

In [162]:
def generate_forecast():
    idx = 0  # Use first sequence in test data
    max_idx = test_data.shape[1] - (hist_length + pred_length)
    if max_idx < 0:
        idx = 0
        print("Warning: Test data too short, using first sequence")
    
    # Get history values
    hist_value = torch.tensor(test_data[:, idx:idx+hist_length], dtype=torch.float32).to(device).unsqueeze(0)
    
    # Get actual future values (if available)
    if idx + hist_length + pred_length <= test_data.shape[1]:
        true_future = test_data[:, idx+hist_length:idx+hist_length+pred_length]
    else:
        # Not enough data for true future, use placeholder
        true_future = np.zeros((test_data.shape[0], pred_length))
    
    hist_time = torch.arange(0, hist_length, device=device).unsqueeze(0)
    pred_time = torch.arange(hist_length, hist_length + pred_length, device=device).unsqueeze(0)
    
    try:
        with torch.no_grad():
            samples = model.sample(20, hist_time, hist_value, pred_time)
            samples_np = samples[0].cpu().numpy()  # [series, pred_timesteps, samples]
            
            # Plot one example
            i = 0  # First series
            plt.figure(figsize=(10, 5))
            
            # Make sure dimensions match
            hist_x = np.arange(hist_length)
            pred_x = np.arange(hist_length, hist_length + pred_length)
            
            # Print shapes before processing
            print(f"Raw samples shape: {samples_np.shape}")
            
            # Get samples for this series and calculate statistics
            s_samples = samples_np[i]  # Shape: [pred_timesteps, samples]
            print(f"Series samples shape: {s_samples.shape}")
            
            # Fix the dimension issue by taking only pred_length steps
            if s_samples.shape[0] > pred_length:
                print(f"Truncating forecast from {s_samples.shape[0]} to {pred_length} steps")
                s_samples = s_samples[:pred_length]
            
            median = np.median(s_samples, axis=1)  # Shape: [pred_timesteps]
            lower = np.percentile(s_samples, 10, axis=1)
            upper = np.percentile(s_samples, 90, axis=1)
            
            # Verify shapes before plotting
            print(f"History shape: {hist_value[0, i].shape}, x-axis: {hist_x.shape}")
            print(f"Forecast shape: {median.shape}, x-axis: {pred_x.shape}")
            print(f"True future shape: {true_future[i].shape}")
            
            # Plot with explicit x-axis values
            plt.plot(hist_x, hist_value[0, i].cpu().numpy(), 'k-', label='History', linewidth=2)
            plt.plot(pred_x, true_future[i], 'r-', label='Actual', linewidth=2)
            plt.plot(pred_x, median, 'b-', label='Forecast', linewidth=2)
            plt.fill_between(pred_x, lower, upper, color='b', alpha=0.2)
            
            plt.title(f'Series: {df.columns[i]}')
            plt.axvline(x=hist_length-1, color='gray', linestyle='--')
            plt.legend()
            plt.savefig('forecast.png')
            plt.close()
            
            # Additionally create a figure with only a subset of the history
            # to better see the transition to forecast
            plt.figure(figsize=(10, 5))
            
            # Use only the last 12 steps of history
            short_hist_len = min(12, hist_length)
            short_hist_x = hist_x[-short_hist_len:]
            short_hist_y = hist_value[0, i, -short_hist_len:].cpu().numpy()
            
            plt.plot(short_hist_x, short_hist_y, 'k-', label='History', linewidth=2)
            plt.plot(pred_x, true_future[i], 'r-', label='Actual', linewidth=2)
            plt.plot(pred_x, median, 'b-', label='Forecast', linewidth=2)
            plt.fill_between(pred_x, lower, upper, color='b', alpha=0.2)
            
            plt.title(f'Zoomed view: {df.columns[i]}')
            plt.axvline(x=hist_length-1, color='gray', linestyle='--')
            plt.legend()
            plt.savefig('forecast_zoomed.png')
            plt.close()
            
            return True
    except Exception as e:
        print(f"Error generating forecast: {e}")
        import traceback
        traceback.print_exc()
        return False


In [ ]:
# Train and forecast
print("Starting training...")
train_model()
print("\nGenerating forecasts...")
generate_forecast()

Starting training...
Phase 1 Training (Marginals)
Epoch 1/5, Loss: 209.803632, Valid Batches: 5/5
Epoch 2/5, Loss: 219.432407, Valid Batches: 5/5
Epoch 3/5, Loss: 200.999216, Valid Batches: 5/5
Epoch 4/5, Loss: 204.633322, Valid Batches: 5/5
Epoch 5/5, Loss: 199.382626, Valid Batches: 5/5

Phase 2 Training (Copula)
